### 모델링을 위한 기본 전처리
- SQL(BigQuery)
- Python => Label Encoding, One Hot Encoding 
- 시간 관련 전처리
    - DATETIME에서 MONTH, HOUR, WEEKDAY 등 추출
    - BigQuery vs Python
- BigQuery GIS로 Reverse Geocoding
    - 좌표 => zip_code로 변환
-  Categorical 데이터 전처리`
    - One Hot Encoding
    - Label Encoding
-  Train / Test Split

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
import seaborn as sns
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')

### 시간 관련 전처리
- 1) BigQuery
    - DATETIME_TRUNC 
    - EXTRACT 
        - [EXTRACT 공식 문서](https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions?hl=ko#extract)
    - FORMAT_DATETIME (요일 값을 숫자로 추출 -> 단, 1이 월요일 => Pandas에선 0이 월요일)
        - [FORMAT_DATETIME 공식 문서](https://cloud.google.com/bigquery/docs/reference/standard-sql/datetime_functions?hl=ko#format_datetime)

In [5]:
# 30분 단위 쿼리
# %%time
# query = """
# SELECT
#     *,
#     EXTRACT(MONTH FROM pickup_hour) AS month,
#     EXTRACT(DAY FROM pickup_hour) AS day,
#     CAST(format_datetime('%u', pickup_hour) AS INT64) -1 AS weekday,
#     EXTRACT(HOUR FROM pickup_hour) AS hour,
#     EXTRACT(MINUTE FROM pickup_hour) AS minute,
#     CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend
# FROM (
#     SELECT 
#         CASE WHEN EXTRACT(MINUTE FROM DATETIME_TRUNC(pickup_datetime, minute)) IN (0, 30) THEN 
#            DATETIME_TRUNC(pickup_datetime, hour) ELSE DATETIME_ADD(DATETIME_TRUNC(pickup_datetime, hour), INTERVAL 30 MINUTE) END
#            AS pickup_hour,
#         count(*) AS cnt
#     FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
#     WHERE EXTRACT(MONTH from pickup_datetime) = 1
#     GROUP BY pickup_hour
# )
# ORDER BY pickup_hour
# """

# df_30 = pd.read_gbq(query=query, dialect='standard', project_id='nyc-taxi-demand')

Downloading: 100%|█████████████████████| 1486/1486 [00:00<00:00, 3331.65rows/s]

Wall time: 4.48 s


In [6]:
# df_30.head()

,pickup_hour,cnt,month,day,weekday,hour,minute,is_weekend
0,2015-01-01 00:00:00,629,1,1,3,0,0,0
1,2015-01-01 00:30:00,27683,1,1,3,0,30,0
2,2015-01-01 01:00:00,1119,1,1,3,1,0,0
3,2015-01-01 01:30:00,30588,1,1,3,1,30,0
4,2015-01-01 02:00:00,964,1,1,3,2,0,0


In [12]:
# 한 시간 단위 쿼리 -> 본 프로젝트에서는 한 시간 간격으로 택시 운행 빈도를 예측
%time
query = """
SELECT 
    DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    count(*) AS cnt
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
WHERE EXTRACT(MONTH from pickup_datetime) = 1
GROUP BY pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour
"""

df = pd.read_gbq(query=query, dialect='standard', project_id='nyc-taxi-demand')

Wall time: 0 ns


Downloading: 100%|███████████████████████| 744/744 [00:00<00:00, 1978.61rows/s]


In [13]:
df.tail()

,pickup_hour,month,day,weekday,hour,is_weekend,cnt
739,2015-01-31 19:00:00,1,31,5,19,1,32436
740,2015-01-31 20:00:00,1,31,5,20,1,27555
741,2015-01-31 21:00:00,1,31,5,21,1,27477
742,2015-01-31 22:00:00,1,31,5,22,1,29862
743,2015-01-31 23:00:00,1,31,5,23,1,29856


- SQL(BigQuery)로 가능한 부분은 SQL에서 => 로컬 파이썬 보다 성능이 좋을 수 있음. 단, 데이터 용량에 따른 IO 소요 시간을 생각할 것!

### `BigQuery GIS`로 Reverse Geocoding
- 좌표 => zip_code로 변환
- BigQuery의 `bigquery-public-data.geo_us_boundaries.zip_codes` 참고
- ST_CONTAINS 함수로 추출 가능

In [14]:
%%time
base_query = """
WITH base_data AS 
(
  SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
  FROM (
    SELECT *
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE 
        EXTRACT(MONTH from pickup_datetime) = 1
        and pickup_latitude  <= 90 and pickup_latitude >= -90
    ) AS nyc_taxi
  JOIN (
    SELECT zip_code, state_code, state_name, city, county, zip_code_geom
    FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
    WHERE state_code='NY'
    ) AS gis 
  ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT 
    zip_code,
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(format_datetime('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    COUNT(*) AS cnt
FROM base_data 
GROUP BY zip_code, pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour


"""

# base 예측 모델 생성을 위한 데이터프레임 생성
base_df = pd.read_gbq(query=base_query, dialect='standard', project_id='nyc-taxi-demand')

Downloading: 100%|███████████████████| 87020/87020 [00:08<00:00, 9814.46rows/s]

Wall time: 20.7 s


In [15]:
base_df.tail()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt
87015,11233,2015-01-31 23:00:00,1,31,5,23,1,6
87016,10039,2015-01-31 23:00:00,1,31,5,23,1,3
87017,10173,2015-01-31 23:00:00,1,31,5,23,1,5
87018,11207,2015-01-31 23:00:00,1,31,5,23,1,2
87019,11225,2015-01-31 23:00:00,1,31,5,23,1,11


### Categorical 데이터 전처리
- One Hot Encoding

In [16]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(base_df[['zip_code']])

OneHotEncoder(handle_unknown='ignore')

In [17]:
print(dir(enc))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_X', '_check_n_features', '_compute_drop_idx', '_fit', '_get_feature', '_get_param_names', '_get_tags', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_transform', '_validate_data', '_validate_keywords', 'categories', 'categories_', 'drop', 'drop_idx_', 'dtype', 'fit', 'fit_transform', 'get_feature_names', 'get_params', 'handle_unknown', 'inverse_transform', 'set_params', 'sparse', 'transform']


In [18]:
ohe_output = enc.transform(base_df[['zip_code']]).toarray()  # 인코딩 결과를 데이터프레임 형태(array)로 변환
ohe_output

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
oh_feature_df = pd.concat([base_df, pd.DataFrame(ohe_output, columns='zip_code_'+enc.categories_[0])], axis=1)
oh_feature_df.head()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
0,10037,2015-01-01,1,1,3,0,0,26,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11238,2015-01-01,1,1,3,0,0,95,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10170,2015-01-01,1,1,3,0,0,44,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11221,2015-01-01,1,1,3,0,0,34,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11371,2015-01-01,1,1,3,0,0,76,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- label encoding

In [20]:
# le = LabelEncoder()
# le.fit(base_df['zip_code'])
# base_df['zip_code_le'] = le.transform(base_df['zip_code'])

### Train and Test Split
- Train set : 과거 데이터, Test set : 미래 데이터
- 두 가지 방법
    - 1)  특정 일자를 기준으로 데이터를 Split
    - 2)  특정 기간을 기준으로 데이터를 Split

###  1) 특정 일자를 기준으로 데이터를 분할하는 함수

In [34]:
def split_train_and_test(df, date):
    """
    Dataframe에서 train_df, test_df로 나눠주는 함수
    
    df : 시계열 데이터 프레임
    date : 기준점 날짜
    """
    train_df = df[df['pickup_hour'] <= date]
    test_df = df[df['pickup_hour'] > date]
    return train_df, test_df

### 2) 특정 기간을 기준으로 데이터를 Split
- ex) 최근 1주일 치 자료를 검증용 자료 set으로 지정

In [30]:
def split_train_and_test_period(df, period):
    """
    Dataframe에서 train_df, test_df로 나눠주는 함수
    
    df : 시계열 데이터 프레임
    period : 기간(정수 값, ex) 3 -> 3일)
    """
    criteria = max(df['pickup_hour']) - pd.Timedelta(days=period)  # 기준 일 계산
    train_df = df[df['pickup_hour'] <= criteria]
    test_df = df[df['pickup_hour'] > criteria]
    return train_df, test_df

In [39]:
pd.Timedelta(days=7)

Timedelta('7 days 00:00:00')

In [31]:
train_df, test_df = split_train_and_test_period(oh_feature_df, 7)

In [32]:
train_df.head()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
0,10037,2015-01-01,1,1,3,0,0,26,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11238,2015-01-01,1,1,3,0,0,95,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10170,2015-01-01,1,1,3,0,0,44,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11221,2015-01-01,1,1,3,0,0,34,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11371,2015-01-01,1,1,3,0,0,76,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
test_df.head()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
68051,10027,2015-01-25,1,25,6,0,1,69,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68052,10005,2015-01-25,1,25,6,0,1,88,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68053,10024,2015-01-25,1,25,6,0,1,514,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68054,10065,2015-01-25,1,25,6,0,1,408,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68055,11201,2015-01-25,1,25,6,0,1,156,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
# train / test set 불필요한 행 제거
del train_df['zip_code']
del train_df['pickup_hour']
del test_df['zip_code']
del test_df['pickup_hour']

In [41]:
# train / test set 독립변수 및 종속변수 분리
y_train = train_df.pop('cnt')
x_train = train_df.copy()
y_test = test_df.pop('cnt')
x_test = test_df.copy()

In [42]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(68051, 379) (68051,) (18969, 379) (18969,)
